In [9]:
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
import pickle
from scipy.interpolate import splprep, splrep, splev
%matplotlib inline
matplotlib.rcParams.update({'font.size': 14})

# open image files
d = 'sample17_batch3_IJ_stack_ok'

files = []
for file in glob.glob('image_stacks/'+d+'/*.ti*'):
    files.append(file)

files.sort()

im = np.array(Image.open(files[0]))

image_stack = np.zeros([len(files),np.shape(im)[0],np.shape(im)[1]])
for i in range(len(files)):
    im = np.array(Image.open(files[i]))
    image_stack[i,:,:] = im

# load the clicks
file_name = 'data_files/'+d+'_shell_leaf_boundaries_d1.dat'
f = open(file_name,'r')
[background_shell,shell_leaf,fs] = pickle.load(f)
f.close()

frames = [int(f[-8:-4]) for f in fs]

# do the periodic spline interpolation in the x-y frame and collect 1000 points on each frame
points = np.zeros([20,3,1000])
for f in range(len(frames)):

    boundary = np.array(shell_leaf[f])
    if len(boundary)>0:
        boundary = np.append(boundary,boundary[0,:][None,:],axis=0)

        center = [np.sum(boundary[:,0])/len(boundary[:,0]),np.sum(boundary[:,1])/len(boundary[:,1])]

        tck, u = splprep(boundary.T, u=None, s=0.0, per=1) 

        u_new = np.linspace(u.min(), u.max(), 1000)
        x_new, y_new = splev(u_new, tck, der=0)
        
        phi = np.arctan2(y_new-center[1], x_new-center[0])/np.pi
        indx = np.where(phi == np.max(phi))[0]
                
        x_shift = np.append(x_new[indx:],x_new[:indx])
        y_shift = np.append(y_new[indx:],y_new[:indx])
        z_shift = np.zeros(len(x_new))+frames[f]
        
        points[f] = np.array([x_shift,y_shift,z_shift])
        
        #plt.figure(figsize=(6,6))
        #plt.imshow(image_stack[frames[f],:,:].T,cmap='Greys_r')
        #plt.plot(x_new,y_new,'r')
        #plt.plot(boundary[:,0],boundary[:,1],'wo')
        #plt.xlim([0,np.shape(image_stack[frames[f],:,:])[0]])
        #plt.ylim([0,np.shape(image_stack[frames[f],:,:])[1]])
        #plt.show()
        #plt.close()
        

# do the vertical interpolation and collect the tcks for splev
splev_x = []
splev_y = []
for i in range(1000):
    x = points[:,0,i]
    y = points[:,1,i]
    z = points[:,2,i]
    tckx = splrep(z[x>0],x[x>0])
    splev_x.append(tckx)
    
    tcky = splrep(z[x>0],y[x>0])
    splev_y.append(tcky)
    
# go through the frames and plot the interpolated contours
f = 0
for i in range(np.shape(image_stack)[0]):
    if i%50 == 0:
        print i
    
    plt.figure(figsize=(6,6))
    plt.axis('equal')
    plt.title('boundary contours')
    plt.xlabel('pixel coord.')
    plt.ylabel('pixel coord.')
    plt.imshow(image_stack[i,:,:].T,cmap='Greys_r',vmin=10000,vmax=33000)
        
    boundary = np.array(shell_leaf[f])
    if len(boundary) > 0:
        x_cont = np.zeros(1000)
        y_cont = np.zeros(1000)
        for j in range(1000):
            tck_x = splev_x[j]
            tck_y = splev_y[j]
            x_cont[j] = splev(i,tck_x)
            y_cont[j] = splev(i,tck_y)

        plt.plot(x_cont,y_cont,'r')

        if i == frames[f]-1:
            boundary = np.array(shell_leaf[f])
            plt.plot(boundary[:,0],boundary[:,1],'wo')
        if i == frames[f]:
            boundary = np.array(shell_leaf[f])
            plt.plot(boundary[:,0],boundary[:,1],'wo')
        if i == frames[f]+1:
            boundary = np.array(shell_leaf[f])
            plt.plot(boundary[:,0],boundary[:,1],'wo')
            f = f + 1

    plt.xlim([0,np.shape(image_stack[i,:,:])[0]])
    plt.ylim([0,np.shape(image_stack[i,:,:])[1]])
    plt.savefig('animation_border/interpolate/frame'+str(i).zfill(4)+'.png',dpi=100)
    plt.close()
    



/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:59: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:60: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
